In [18]:
#Bibliotecas
import pandas as pd
import re
import datetime as dt
from textwrap import wrap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
import sklearn

#Importando os dados
#Planilha com descrição do produto comprado
with open('data/RELATORIO_VENDAS.txt', 'rb') as file:
    file_lines = file.readlines(1000)   
pd.read_csv('data/RELATORIO_VENDAS.txt', sep = '\t', decimal = '.', encoding = 'latin-1')
pd_relatorio_produto = pd.read_fwf('data/RELATORIO_VENDAS.txt',  encoding = 'latin-1', skiprows=5, skipfooter=4)


with open('data/relatoria_vendas1.txt', 'rb') as file:
    file_lines = file.readlines(1000)
    
pd.read_csv('data/relatoria_vendas1.txt', 
                       sep = '\t', 
                       decimal = '.', 
                       encoding = 'latin-1')

pd_relatorio_cliente = pd.read_fwf('data/relatoria_vendas1.txt',  encoding = 'latin-1', skiprows=5, skipfooter=4)



#Renomeando as colunas
novo_nome_produto = {
    '000000534': 'Nota',
    '01/01/2020': 'Data',
    'Planner 2020 Personalizado': 'Descricao',
    '1,00': 'Und',
    '189,00': 'Preco',
    '0,00': 'Custo'
}

pd_relatorio_produto = pd_relatorio_produto.rename(columns=novo_nome_produto)

novo_nome_cliente = {
    '000000533': 'Nota',
    '01/01/2020': 'Data',
    'Cristiane Brito': 'Cliente',
    '189,00': 'Produtos R$',
    '0,00': 'Serviços R$',
    '27,80': 'Frete R$',
    '9,45': 'Desconto R$',
    '0,00.1': 'Outras R$',
    '207,35': 'Total R$',
    '0,00.2': 'ICMS R$'
}

# Rename colums
pd_relatorio_cliente = pd_relatorio_cliente.rename(columns=novo_nome_cliente)




#
mask_nota_534 = pd_relatorio_cliente['Nota'] > 534
pd_relatorio_cliente = pd_relatorio_cliente[mask_nota_534]
pd_relatorio_cliente


#Alterando type columns
def limpar_numero(value):
    pattern = "[^0-9]"
    value_novo = re.sub(pattern, '', str(value))
    return int(value_novo)


pd_relatorio_cliente['Produtos R$'] = pd_relatorio_cliente['Produtos R$'].map(limpar_numero)
pd_relatorio_cliente['Frete R$'] = pd_relatorio_cliente['Frete R$'].map(limpar_numero)
pd_relatorio_cliente['Desconto R$'] = pd_relatorio_cliente['Desconto R$'].map(limpar_numero)
pd_relatorio_cliente['Total R$'] = pd_relatorio_cliente['Total R$'].map(limpar_numero)
pd_relatorio_cliente['Data'] = pd.to_datetime(pd_relatorio_cliente['Data'])

#Atualizando valores colunas
pd_relatorio_cliente['Produtos R$'] = pd_relatorio_cliente['Produtos R$'] / 100
pd_relatorio_cliente['Frete R$'] = pd_relatorio_cliente['Frete R$'] / 100
pd_relatorio_cliente['Desconto R$'] = pd_relatorio_cliente['Desconto R$'] / 100
pd_relatorio_cliente['Total R$'] = pd_relatorio_cliente['Total R$'] / 100

#Drop columns
pd_relatorio_cliente = pd_relatorio_cliente.drop(columns=['Serviços R$', 'Outras R$', 'ICMS R$'])


#Merge
pd_vendas = pd.merge(pd_relatorio_produto, pd_relatorio_cliente, on="Nota", how="inner")

# Drop columns
# pd_vendas = pd_vendas.drop(columns=['Data_y'])

# Rename columns
pd_vendas = pd_vendas.rename(columns={'Data_x': 'Data'})

# Reordering columns
pd_vendas = pd_vendas[['Nota', 'Data', 'Cliente', 'Descricao', 'Und', 'Preco', 'Frete R$',
                     'Desconto R$', 'Total R$']]

# Alterando Type
pd_vendas['Und'] = pd_vendas['Und'].astype(str).str.replace(',', '.').astype(float)
pd_vendas['Preco'] = pd_vendas['Preco'].astype(str).str.replace(',', '.').astype(float)

# Convertendo colunas
if pd_vendas['Frete R$'].dtype != float:
    pd_vendas['Frete R$'] = pd_vendas['Frete R$'].astype(str).str.replace(',', '.').astype(float)

pd_vendas['Desconto R$'] = pd_vendas['Desconto R$'].astype(str).str.replace(',', '.').astype(float)
pd_vendas['Total R$'] = pd_vendas['Total R$'].astype(str).str.replace(',', '.').astype(float)

# Drop columns
# pd_vendas = pd_vendas.drop(columns=['Data_y'])

# Rename columns
pd_vendas = pd_vendas.rename(columns={'Data_x': 'Data'})

# Reordering columns
pd_vendas = pd_vendas[['Nota', 'Data', 'Cliente', 'Descricao', 'Und', 'Preco', 'Frete R$',
                     'Desconto R$', 'Total R$']]

# Alterando Type
pd_vendas['Und'] = pd_vendas['Und'].astype(str).str.replace(',', '.').astype(float)
pd_vendas['Preco'] = pd_vendas['Preco'].astype(str).str.replace(',', '.').astype(float)

# Convertendo colunas
if pd_vendas['Frete R$'].dtype != float:
    pd_vendas['Frete R$'] = pd_vendas['Frete R$'].astype(str).str.replace(',', '.').astype(float)

pd_vendas['Desconto R$'] = pd_vendas['Desconto R$'].astype(str).str.replace(',', '.').astype(float)
pd_vendas['Total R$'] = pd_vendas['Total R$'].astype(str).str.replace(',', '.').astype(float)


pd_vendas.to_csv('relatorio_vendas_cliente.csv', index=False)



C:\Users\Matheus\AppData\Local\Temp\ipykernel_8700\1811317790.py:80: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd_relatorio_cliente['Data'] = pd.to_datetime(pd_relatorio_cliente['Data'])


In [23]:
#Segunda parte do projeto. 
#Machine Learning Recomendação de produtos para antigos clientes

# Matriz cliente produto
matriz = pd_vendas.pivot_table(index='Cliente', columns='Descricao', values='Und', aggfunc='sum', fill_value=0)


sca = StandardScaler()

#Pipeline
#matriz['Und'] = matriz['Und'].str.replace(',', '.').astype(float)
sca = StandardScaler()
matriz_normalizada = pd.DataFrame(sca.fit_transform(matriz.values), index=matriz.index, columns=matriz.columns)
pca = TruncatedSVD(4)
pca.fit(matriz)

matriz_pca = pca.transform(matriz)
matriz_recomendacao = pd.DataFrame(pca.inverse_transform(matriz_pca), index=matriz_normalizada.index, columns=matriz_normalizada.columns)

recomendacao = matriz_recomendacao.reset_index().melt(id_vars='Cliente').query('value>0.1') #melt joga coluna para as linhas
recomendacao['value'] = recomendacao['value'].round(2)

pd_produto_cliente = pd_vendas.groupby(['Cliente', 'Descricao'])['Und'].sum().reset_index()
pd_final = pd.merge(pd_produto_cliente, recomendacao, on=['Cliente', 'Descricao'], how='right').fillna(0) 
pd_final.to_csv('indicacao.csv', index=False, decimal=',', sep=';')